In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import seaborn as sns\n",
    "\n",
    "from pyspark.sql import SparkSession\n",
    "from pyspark.sql import functions as f\n",
    "\n",
    "spark = SparkSession.builder.appName(\"DataExploration\").getOrCreate()\n",
    "\n",
    "pd.options.display.max_columns = None\n",
    "pd.options.display.max_rows = 250\n",
    "pd.options.display.max_colwidth = 150\n",
    "sns.set(color_codes=True)\n",
    "\n",
    "schema = \"polarity FLOAT, id LONG, date_time STRING, query STRING, user STRING, text STRING\"\n",
    "\n",
    "IN_PATH_1 = \"/home/jovyan/data-sets/sentiment-140-training-data/testdata.manual.2009.06.14.csv\"\n",
    "IN_PATH_2 = \"/home/jovyan/data-sets/sentiment-140-training-data/training.1600000.processed.noemoticon.csv\"\n",
    "OUT_PATH = \"/home/jovyan/data-sets/sentiment-140-training-data/RAW\"\n",
    "\n",
    "spark_reader = spark.read.schema(schema)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "test_data = spark_reader.csv(IN_PATH_1, header=False).cache()\n",
    "training_data = spark_reader.csv(IN_PATH_2, header=False).cache()\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "498"
      ]
     },
     "execution_count": 13,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "test_data.count()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "1600000"
      ]
     },
     "execution_count": 14,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "training_data.count()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {},
   "outputs": [],
   "source": [
    "df = training_data.select(\"polarity\").na.drop()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<matplotlib.axes._subplots.AxesSubplot at 0x7f126c8cf630>"
      ]
     },
     "execution_count": 19,
     "metadata": {},
     "output_type": "execute_result"
    },
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAXMAAAEBCAYAAACKUEVYAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDMuMC4zLCBodHRwOi8vbWF0cGxvdGxpYi5vcmcvnQurowAAGnNJREFUeJzt3XtsVNedB/DvvTMe24MfMx6P3y/Mwxhw4hQSp4G0kZGKmriE7kpNZLF/NA82ZRtFrVDpQgobSBo5YZUmAgkldHeVKKLKalsSoLskhLxwKCEJED/ANviNx9gej7E9Y3vsmbt/GNMQ28yMPXfuzJnvR4oU39zx/Z3r3K+vzz33HElRFAVERBTRZK0LICKi+WOYExEJgGFORCQAhjkRkQAY5kREAmCYExEJgGFORCQAhjkRkQAY5kREAmCYExEJgGFORCQAhjkRkQAY5kREAtCH4iAOhxNeb3hNzmixJMBuH9a6DE1F+zlg+6O7/UD4ngNZlmA2LwjoMyEJc69XCbswBxCWNYVatJ8Dtj+62w+Icw7YzUJEJACGORGRABjmREQCYJgTEQmAYU5EJACGORGRABjmREQCCMk483A05HLDOTYxbXtsjB56/oojEs6EFxgbv/Waj3O5Naom+KI2zEdGJ3D24rVp2+8uToc+NmpPC5GwxsanX/M/XJUHSaN6go33oEREAmCYExEJgGFORCQAvzqHx8bG8Pvf/x6nT59GbGwsSktLsWfPHrVrIyIiP/kV5i+//DJiY2Nx/PhxSJKEvr4+tesiIqIA+Axzp9OJw4cP45NPPoEkTT73TU1NVb0wIiLyn88w7+jogMlkwr59+3DmzBksWLAAzzzzDFavXu33QSyWhHkVqYaefhcSE+KmbTcaY2FNMWpQkTas1kStS9AU2x897VdmueZFOQc+w3xiYgIdHR1Yvnw5tm3bhgsXLuCpp57CBx98gIQE/0Labh8OvwngdToMDY9O2+xyjaHX49GgoNCzWhPR2zukdRmaYfujq/2usYkZr/lwPAeyLAV8E+xzNEtWVhb0ej0qKioAAHfeeSfMZjNaWlrmViUREQWdzzBPSUlBWVkZqqurAQAtLS2w2+3Iz89XvTgiIvKPX6NZnnvuOWzfvh1VVVXQ6/V46aWXkJSUpHZtRETkJ7/CPDc3F2+99ZbatRAR0RzxDVAiIgEwzImIBMAwJyISAMOciEgADHMiIgEwzImIBMAwJyISAMOciEgADHMiIgEwzImIBMAwJyISAMOciEgADHMiIgEwzImIBMAwJyISAMOciEgADHMiIgEwzImIBMAwJyISAMOciEgADHMiIgEwzImIBMAwJyISAMOciEgAen92Ki8vh8FgQGxsLABg69atuP/++1UtjIiI/OdXmAPAa6+9hqVLl6pZCxERzRG7WYiIBOD3nfnWrVuhKApWrVqFX//610hKSlKzLiIiCoCkKIriayebzYbMzEy43W688MILcDqd2Lt3byjqU01PvwtfN/RM2/69ojSkpRg1qIiI1DTTNS/S9e7XnXlmZiYAwGAwoLKyEr/4xS8COojdPgyv1+fvjNDS6TA0PDpts8s1hl6PR4OCQs9qTURv75DWZWiG7Y+u9rvGJma85sPxHMiyBIslIbDP+NrB5XJhaGiysYqi4K9//SuKi4vnViEREanC55253W7H008/DY/HA6/Xi0WLFmHXrl2hqI2IiPzkM8xzc3Nx+PDhUNRCRERzxKGJREQCYJgTEQmAYU5EJACGORGRABjmREQCYJgTEQmAYU5EJACGORGRABjmREQCYJgTEQmAYU5EJACGORGRABjmREQCYJgTEQmAYU5EJACGORGRABjmREQCYJgTEQmAYU5EJACGORGRABjmREQCYJgTEQmAYU5EJACGORGRAAIK83379qGoqAiNjY1q1UNERHPgd5jX1dXh/PnzyMrKUrMeIiKaA7/C3O12Y/fu3di1axckSVK7JiIiCpBfYf7qq69iw4YNyM3NVbseIiKaA72vHc6dO4eamhps3bp1zgexWBLm/Fm19PS7kJgQN2270RgLa4pRg4q0YbUmal2Cptj+6Gm/Mss1L8o58BnmZ8+eRXNzM9atWwcA6O7uxuOPP44XX3wRa9eu9esgdvswvF5lfpUGm06HoeHRaZtdrjH0ejwaFBR6VmsienuHtC5DM2x/dLXfNTYx4zUfjudAlqWAb4J9hvnmzZuxefPmm1+Xl5fjwIEDWLp0aeAVEhGRKjjOnIhIAD7vzL/r5MmTatRBRETzwDtzIiIBMMyJiATAMCciEgDDnIhIAAxzIiIBMMyJiATAMCciEgDDnIhIAAxzIiIBMMyJiATAMCciEgDDnIhIAAxzIiIBMMyJiATAMCciEgDDnIhIAAxzIiIBMMyJiATAMCciEgDDnIhIAAxzIiIBMMyJiATAMCciEgDDnIhIAHp/dtqyZQs6OzshyzKMRiN+97vfobi4WO3aiIjIT36FeVVVFRITEwEAJ06cwPbt2/GXv/xF1cKIiMh/fnWzTAU5AAwPD0OSJNUKIiKiwPl1Zw4AO3bsQHV1NRRFwcGDB9WsiYiIAiQpiqIE8oHDhw/j2LFjeOONN9SqKSR6+l34uqFn2vbvFaUhLcWoQUVEpKaZrnmRrne/78ynbNy4ETt37oTD4YDZbPbrM3b7MLzegH5nqE+nw9Dw6LTNLtcYej0eDQoKPas1Eb29Q1qXoRm2P7ra7xqbmPGaD8dzIMsSLJaEwD7jawen0wmbzXbz65MnTyI5ORkmkynwComISBU+78xHRkbwzDPPYGRkBLIsIzk5GQcOHOBDUCKiMOIzzFNTU/HOO++EohYiIpojvgFKRCQAhjkRkQAY5kREAmCYExEJgGFORCQAhjkRkQAY5kREAmCYExEJgGFORCQAhjkRkQAY5kREAmCYExEJgGFORCQAhjkRkQAY5kREAmCYExEJgGFORCQAhjkRkQAY5kREAmCYExEJgGFORCQAhjkRkQAY5kREAmCYExEJQO9rB4fDgd/85jdob2+HwWBAfn4+du/ejZSUlFDUR0REfvB5Zy5JEp544gkcP34cR44cQW5uLvbu3RuK2oiIyE8+w9xkMqGsrOzm16Wlpejq6lK1KCIiCkxAfeZerxeHDh1CeXm5WvUQEdEc+Owz/7Y9e/bAaDRi06ZNAR3EYkkIaP9Q6Ol3ITEhbtp2ozEW1hSjBhVpw2pN1LoETbH90dN+ZZZrXpRz4HeYV1VVoa2tDQcOHIAsBzYIxm4fhterBFycqnQ6DA2PTtvsco2h1+PRoKDQs1oT0ds7pHUZmmH7o6v9rrGJGa/5cDwHsiwFfBPsV5i/8sorqK2txeuvvw6DwTCn4oiISD0+w7ypqQkHDhxAQUEBHn30UQBATk4O9u/fr3pxRETkH59hvmTJEjQ0NISiFiIimiO+AUpEJACGORGRABjmREQCYJgTEQmAYU5EJACGORGRAKI2zHscLrjHo+NNTyK6lderwDE0/W3QSBbQ3CwiGBmbwOHPWnDiqw6YE2Px43vzoZMlrcsiohD6vLYbzV2DuNo3gsp1i5GSNH3OlkgTVXfm9uujePbgGZz4sgN3Lraif3AM55t6tS6LiEKoxTaI5q5BZFqMqL3Shx0Hz+BSm0PrsuYtqsL8w686Meh04183rcLTPyvF0txk1LU4YLM7tS6NiEJgeGQcf6u7BqspDutW5WD3P9+HBXF6HPm8VevS5i1qwnx8woNTNTaULknF4pxkAMDqZWlIXmBA9Tfd8Hi9GldIRGr7W103oABr78iELEuwmuLxQGk2LrY50N3v0rq8eYmaMP+qoRfDI+N4oDT75ja9TsaqIitcYxOw2SP7B0lEtzcwPIauPheWLzQj0fj32V/vvyMTOlnCJ+evaljd/EVNmH98vgtppngUF5hv2Z6ZakSMXkZbd/jNaUxEwXPhch8AID/j1sUokhNicdeSVFTXdGN8InJHuEVFmF/tc6KxYwA/LM2CLN06ckUny8hNS0BHTxguoEFEQXOhqQ/JCQaYEmKn/bcH7srG8Mg4vmyI3AERURHmn13ogk6WsKYkc8b/npeeAPe4N+L7zIhoZtedbly+eh356TMvEbcs34w0czw+OR+5i9VHRZifv9yHFQtTkLRg5lWSslIXQK+T2NVCJKhzjb1QFCA/Y+al2GRJwr3L09HUOQDn6HiIqwsO4cO8b2AEPY4RrChImXUfvU5GjnWyq8XDrhYi4XzZ0AOrKX7GLpYpywtSoCiI2DHnwod5XWs/AGD5wtnDHJh8KDLq9uDK1euhKIuIQmR4ZByX2gZQuiQVkjT7296FWUmIM+hQ18owD0t1rQ6YEgzIshhvu19W6gLIEnDxRvgTkRjqW/vhVRSULLLcdj+9TsayPDPqWyIzA4QOc69XwcXWfqwoSLntb2QAiNHLSDXFo6mTd+ZEIrnUPoBYgw55szz8/LblBWb0DIygd2AkBJUFl9Bh3nZtCM7RCZ9dLFPSU4zouDaEkbEJlSsjolBpaHdgaY7Jrwn1VtzIiroI/Atd6DCvn+ovv83Dz29LN8fDqwCX2W9OJITrTjdsdheW5Zn82j8jxQhzYmxEdrUIHeZ1Lf3IsSYgeZYhid9lNcVDJ0u41B6ZD0CI6FYNN67lpX6GuSRJWFGQgottjoh7iVDYMB8b9+Dy1etYsdDse+cbYvQy8jMS0dA+oGJlRBQqDTf6y2d7WWgmyxea4RydQGuEvXcibJg3dw1iwqOgON//MAeAJTnJaLUNYdTNfnOiSNfQMYAlOcnQ6/yPuuK8ycxo7IismzqfLayqqkJ5eTmKiorQ2NgYipqCoqHdAUkCFmf79+fVlMU5JngVBZc5qoUoog063ejqc2JZXmA3dMkJsUhPMYoX5uvWrcPbb7+N7OxsX7uGlcaOAeSlJcIYF9jKeAuzkm70m0fWD5KIbtVwI4yLcgO7oZv8TDIaOwYiqt/cZ5ivXr0amZkzT1AVriY8XlzpGsTSOfwQY2N0WJiZdPPBCRFFpkvtDsTG6KZNeeuPolwzXGMT6OwdVqEydYRkQWeLZebJbdRS32LH+IQXd6/MgNU68w+yp9+FxITpi7gajbG4a1ka/vzRZSQkxSM+Vuw1r2c7P9GC7Re3/Ve6BrGi0ILMjMmVxZRZrvmZzsG9pTq8cbQeXY5RrFqZpXqtwRCSpLLbQztX+Bc1k9NYpifFord3lifSOh2GhkenbXa5xpCbaoTHq+BvFzqxcuHtXwGOZFZr4uznJwqw/eK2f9DpRnv3EO4ust5so2tsYsZrfqZzIAGwJMXh64vduHeZVe1yp5FlKeCbYCFHszR0DCA7dcEtS0MFYnF2MnSyxCGKRBFq6uFlUYAPP79taa4JjR0DUJTI6DcXLsw9Xi8ud16fU3/5lDiDHgUcb04UsRraB2CIkVEwh/7yKUV5Jgy6xiNm0RqfYf7888/jBz/4Abq7u/Hzn/8cDz30UCjqmrP2a8MYdXtQ5OcbX7MpyjOjxTaIMXfkrglIFK0udTiwJMcU0Pjy75oaBdMQIUMUfbb02Wefxaeffor6+npUV1fj2LFjoahrzqbuppfkzC/Ml+WZ4PEqnKeFKMIMudy42uv0ez6W2aSZ45G8wBAxf6EL181ysc2BTMvkZDnzsTgnGbIkoaGDQxSJIsnN/vLcufeXA5PztBTnm3GxzRER/eZChfmEx4uGDgeW5/s3S+LtxBn0KMhM5MtDRBHm0lR/eeb8h10WF5gx6HTjap8zCJWpS6gwb+4ahHvci+KC+f1GnlKUZ0JLF/vNiSJJQ7sDS7IDm49lNlM3hvURsJScUGFe39oPScK8+8qmLMszw+NV0NTJu3OiSDDodKOz1zmvIYnfZkmOQ7o5PiKWkxQszB0oyEiCMS4mKN9vae7k0/Ca5vD/QRIRUNtiBwCsLJx/V+uU4oIUXOoYwITHG7TvqQZhwnxkbALNXYNYHqQuFmBynpZleSbUNNuD9j2JSD01zf1IMsb4td6nv5bnmzHm9qDFNhi076kGYcK8oWMAXkXxe4k4f5UUWtDd74rIBV6JoonXq6C22Y6VhRbIPhZwD8SyfDMkhH+/uTBhXt/ajxi9jMXZSUH9viWLJudm4d05UXhrsQ3COTqBksLgzqeUEB+DvIzEm2sKhythwryupR9LcpIRo9cF9fumm+ORmhyHmisMc6JwVtNshyQBKxYG969zAFheYEZz1yBco+G7ApkQYX61zwmb3YW7lgR/djNJklCyyIKL7Q6MT4T3AxCiaFbT3I/CzCQkxAdnAMS33bXYCo9XwYXLfUH/3sEiRJh/eakHEoBVRepMVVlSaIF73ItGDlEkCkuDLjdabYNB72KZUpidBHNiLM5e6lHl+weDMGG+JNcEU8L8XuGfTXGeGXqdhG8us6uFKBzVNtuh4O/PuIJNliSsLkpDbYs9bLtaIj7Mr/Y5cbXPibuXpal2jFiDDiWFFpy5eC3sx5oSRaPTtd2wJMXNaYk4f929LA0TnvDtaon4MFe7i2XK2pJMDDrdqOULRERhpX9wFPWtDqwpyQjqkMTvCveuFiHCXM0ulikliyxIMsbgVI1N1eMQUWCqa7uhAFhTou7C89/uahkZC7+ulogO887eYdW7WKbodTLuXZGBC5f7MOhyq348IvJNURRU19iwLM8Eqyle9eNNdbV83dir+rECFdFhfvxMOwwxMu4pVj/MAWDtHZnweBWcqbsWkuMR0e01dV5Hj2NE9bvyKYXZSci0GHH8i/awm+M8YsO8b2AEp+uu4Yd3Zs954eZA5VgTUJCRiM++sYXdD5IoGp2qsSHWoMPqotDc0MmShIe+n4/OXicuhNmLhBEb5v/3RTskCVh/T25Ij/vAXdno7B3G143h+USbKFpc63fhdG03vr8iA7GG4L75fTv3FKcjNTkOxz5vDaubuogM8+vDY/j0gg1rSjKQkhQX0mOvKclAVuoC/PdHl/lGKJGG3vnoMvR6GQ+vKQjpcfU6GT8uy8OVrsGwWh80IsP86Ok2eLxe/LgsP+TH1skyHi1fjJ6BEXz4VWfIj09Ek2v9nmvqQ8X385Gs8ki2may9IxNJCww4/FkzvN7wuDuPuDA/19iLD7/qxAN3ZSM9xahJDSsLLSgptODI560c2UIUYl6vgj992ARLUhx+dHdou1mnxOh1+McfFKKx8zreq27RpIbviqgwv+Zw4eCxiyjISMSj5Ys1reWR8sVwj3uw7881XCOUKEQURcGbxy+ho2cYPytfHPRZUgOx9o5MrFmZgSPVrWExRXbEhHnf9RHs+3MNZAnYsnGlpj9EAMhKXYDNG1bgytXreO1/vsH4BAOdSE2KouDQiSZ8esGGivsKQvJ+ye1IkoRN64uQbU3A6+/V4UrXdU3r8SvMW1pa8Mgjj2D9+vV45JFH0NraqnJZf6coCk7XdmPXf3wB+/VRPPXwSqSG4OUAf9y9LA2PPViMi20O/PufzqOjZ1jrkoiE5Bgaw8Gj9TjxVSd+dHcufnr/Qq1LAjC5tOS//MNKxBn0ePGtr/HuqRZ4vNoMjND7s9OuXbtQWVmJhx9+GO+++y527tyJN998U9XCbHYnPjnfhQuX+3DNMYLFOcl4smJ5SN7yCsSakkxIEnDoRBP+7T+/wH0rM1BWnI4luSbExmj71wNRJPN4vWixDeFc0+RzMo9HQcV9Bfjp/QshqTgHS6DSzUY899g9ePuDRrx7qgUfnbuKOxZZsGZlBorygrcmsS+S4mOgpN1ux/r163HmzBnodDp4PB6UlZXh/fffR0qKfyt6OBzOgJ/4/tf/XkJDhwOLspNxR6EFq4vSIMvB+wEqOh3O1HRN237n4lTEz2HMqmt0Ah9+3YnPa7vh8Xih00mwmuJhSohFQnwMDHoZet2Nf/QSdDP9zzjDthlbPNNHZ97ztoxGA1w3HuAqCPCJvLq73/iQuqME4uINGAnwAXbgFQX2ibmdp7ntbowzwDXqu/2hGEvtVYDxCS/Gxz0YGhnHwLAbfQMjGBv3QAKwclEqHrw3D5Z5DEUecXumzXhYVpIFyRO8LtL6NgfONfaioX0AhhgZO/5p1Zx+8ciyBLN5QUCf8RnmtbW12LZtG44dO3Zz24MPPoiXX34ZK1asCLhIIiIKvoh5AEpERLPzGeaZmZm4du0aPDf+FPF4POjp6UFmZmgmtiEiIt98hrnFYkFxcTGOHj0KADh69CiKi4v97i8nIiL1+ewzB4ArV67gt7/9LQYHB5GUlISqqioUFhaGoj4iIvKDX2FOREThjQ9AiYgEwDAnIhIAw5yISAAMcyIiAURlmGs5cZjWqqqqUF5ejqKiIjQ2NmpdTsg5HA48+eSTWL9+PX7yk5/gl7/8Jfr7+7UuK+S2bNmCDRs2YOPGjaisrMTFixe1LkkT+/btE+ZaiMown5o47Pjx46isrMTOnTu1Lilk1q1bh7fffhvZ2dlal6IJSZLwxBNP4Pjx4zhy5Ahyc3Oxd+9ercsKuaqqKrz33ns4fPgwHnvsMWzfvl3rkkKurq4O58+fR1ZWltalBEXUhbndbkd9fT0qKioAABUVFaivr4+au7PVq1dH9du7JpMJZWVlN78uLS1FV9f0CddEl5iYePPfh4eHw2oWwlBwu93YvXs3du3aJUzb/ZoCVyQ2mw3p6enQ6SZnRtTpdEhLS4PNZuNbrVHG6/Xi0KFDKC8v17oUTezYsQPV1dVQFAUHDx7UupyQevXVV7Fhwwbk5mqz7Jwaou7OnGjKnj17YDQasWnTJq1L0cQLL7yAjz/+GL/61a/w0ksvaV1OyJw7dw41NTWorKzUupSgirow58RhBEz2Gbe1teEPf/gDZDnqLoNbbNy4EWfOnIHD4dC6lJA4e/YsmpubsW7dOpSXl6O7uxuPP/44Tp06pXVp8xJ1/xdz4jB65ZVXUFtbi/3798NgMGhdTsg5nU7YbLabX588eRLJyckwmUwaVhU6mzdvxqlTp3Dy5EmcPHkSGRkZ+OMf/4i1a9dqXdq8ROXcLNE8cdjzzz+P999/H319fTCbzTCZTLcsPCK6pqYmVFRUoKCgAHFxk6vW5OTkYP/+/RpXFjp9fX3YsmULRkZGIMsykpOTsW3btqhdbKa8vBwHDhzA0qVLtS5lXqIyzImIRBN13SxERCJimBMRCYBhTkQkAIY5EZEAGOZERAJgmBMRCYBhTkQkAIY5EZEA/h9J8jS2eAG7PwAAAABJRU5ErkJggg==\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "sns.distplot(df.toPandas())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>polarity</th>\n",
       "      <th>count</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>4.0</td>\n",
       "      <td>800000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>0.0</td>\n",
       "      <td>800000</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "   polarity   count\n",
       "0       4.0  800000\n",
       "1       0.0  800000"
      ]
     },
     "execution_count": 21,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "training_data.groupBy(\"polarity\").count().toPandas()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "1600000"
      ]
     },
     "execution_count": 23,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "training_data.select(\"text\").na.drop().count()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 24,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>summary</th>\n",
       "      <th>polarity</th>\n",
       "      <th>id</th>\n",
       "      <th>date_time</th>\n",
       "      <th>query</th>\n",
       "      <th>user</th>\n",
       "      <th>text</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>count</td>\n",
       "      <td>1600000</td>\n",
       "      <td>1600000</td>\n",
       "      <td>1600000</td>\n",
       "      <td>1600000</td>\n",
       "      <td>1600000</td>\n",
       "      <td>1600000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>mean</td>\n",
       "      <td>2.0</td>\n",
       "      <td>1.9988175522956276E9</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>4.325887521835714E9</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>stddev</td>\n",
       "      <td>2.000000625000293</td>\n",
       "      <td>1.935760736226725E8</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>5.1627332184548904E10</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>min</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1467810369</td>\n",
       "      <td>Fri Apr 17 20:30:31 PDT 2009</td>\n",
       "      <td>NO_QUERY</td>\n",
       "      <td>000catnap000</td>\n",
       "      <td>exhausted</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>25%</td>\n",
       "      <td>0.0</td>\n",
       "      <td>1956925281</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>32508.0</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>5</th>\n",
       "      <td>50%</td>\n",
       "      <td>0.0</td>\n",
       "      <td>2002102801</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>130587.0</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6</th>\n",
       "      <td>75%</td>\n",
       "      <td>4.0</td>\n",
       "      <td>2177050908</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>1100101.0</td>\n",
       "      <td>None</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>7</th>\n",
       "      <td>max</td>\n",
       "      <td>4.0</td>\n",
       "      <td>2329205794</td>\n",
       "      <td>Wed May 27 07:27:38 PDT 2009</td>\n",
       "      <td>NO_QUERY</td>\n",
       "      <td>zzzzeus111</td>\n",
       "      <td>�����ߧ�ǿ�����ж�؜��� &amp;lt;&amp;lt;----I DID NOT KNOW I CUD or HOW TO DO ALL DAT ON MY PHONE TIL NOW. WOW..MY LIFE IS NOW COMPLETE.  JK.</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "  summary           polarity                    id  \\\n",
       "0   count            1600000               1600000   \n",
       "1    mean                2.0  1.9988175522956276E9   \n",
       "2  stddev  2.000000625000293   1.935760736226725E8   \n",
       "3     min                0.0            1467810369   \n",
       "4     25%                0.0            1956925281   \n",
       "5     50%                0.0            2002102801   \n",
       "6     75%                4.0            2177050908   \n",
       "7     max                4.0            2329205794   \n",
       "\n",
       "                      date_time     query                   user  \\\n",
       "0                       1600000   1600000                1600000   \n",
       "1                          None      None    4.325887521835714E9   \n",
       "2                          None      None  5.1627332184548904E10   \n",
       "3  Fri Apr 17 20:30:31 PDT 2009  NO_QUERY           000catnap000   \n",
       "4                          None      None                32508.0   \n",
       "5                          None      None               130587.0   \n",
       "6                          None      None              1100101.0   \n",
       "7  Wed May 27 07:27:38 PDT 2009  NO_QUERY             zzzzeus111   \n",
       "\n",
       "                                                                                                                                text  \n",
       "0                                                                                                                            1600000  \n",
       "1                                                                                                                               None  \n",
       "2                                                                                                                               None  \n",
       "3                                                                                                                          exhausted  \n",
       "4                                                                                                                               None  \n",
       "5                                                                                                                               None  \n",
       "6                                                                                                                               None  \n",
       "7  �����ߧ�ǿ�����ж�؜��� &lt;&lt;----I DID NOT KNOW I CUD or HOW TO DO ALL DAT ON MY PHONE TIL NOW. WOW..MY LIFE IS NOW COMPLETE.  JK.  "
      ]
     },
     "execution_count": 24,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "training_data.summary().toPandas()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [],
   "source": [
    "training_data.repartition(20).write.partitionBy(\"polarity\").csv(OUT_PATH, mode=\"overwrite\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}